In [ ]:
from watchdog.observers import Observer
from watchdog.events import *
import time
import os
import logging
import datetime
import pymysql

class FileEventHandler(FileSystemEventHandler):
    def on_any_event(self, event):
        pass
    
    def on_moved(self, event):
        if event.is_directory:
            print("directory moved from {0} to {1}".format(event.src_path,event.dest_path))
        else:
            print("file moved from {0} to {1}".format(event.src_path,event.dest_path))

    def on_created(self, event):
        if event.is_directory:
            print("directory created:{0}".format(event.src_path))
        else:
            print("file created:{0}".format(event.src_path))
            file_name=os.path.split(event.src_path)[1]
            #f_name = os.path.splitext(file_name)[0]
            #suffix_name = os.path.splitext(event.src_path)[1]
            file_,file_format=file_name.split('.')
            file_date=file_.split('_')[-1]
            if file_format not in ('csv','txt'):
                print('输入错误')
                os.remove('./opt/caiji/sej001'+'/'+file_name)
            else:
                print("file created:{0}".format(event.src_path))
                db = pymysql.connect(host='localhost', user='root', passwd='970110',db='wk',port=3306,charset="utf8",local_infile=1,autocommit=True)
                cursor = db.cursor()
                sql='''
                load data local infile "./opt/caiji/sej001/{}" into table w2 FIELDS TERMINATED BY ',' (id,title,link,comment);
                '''.format(file_name)
                try:
                    start_time=datetime.datetime.now()
                    print('运行开始时间：' + format(start_time))
                    # 执行SQL语句
                    cursor.execute(sql)
                    end_time=datetime.datetime.now()
                    db.commit()
                    print(sql)
                    print('运行结束时间：' + format(end_time))
                    print('运行时长：{:.3f}s'.format((end_time-start_time).total_seconds()))
                    print('影响行数：',cursor.rowcount)
                    print('执行成功！')
                    cursor.close()
                    db.close()
                except pymysql.Error as e:
                    print('异常出错：' + str(e))

    def on_deleted(self, event):
        if event.is_directory:
            print("directory deleted:{0}".format(event.src_path))
        else:
            print("file deleted:{0}".format(event.src_path))
    
    def on_modified(self, event):
        if event.is_directory:
            print("directory modified:{0}".format(event.src_path))
        else:
            print("file modified:{0}".format(event.src_path))        
    
if __name__ == "__main__":
    import time
    
    observer = Observer()
    event_handler = FileEventHandler()
    observer.schedule(event_handler, "./opt/caiji/sej001", True)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()


file created:./opt/caiji/sej001\d5.csv
file created:./opt/caiji/sej001\d5.csv
运行开始时间：2020-07-28 10:15:31.501839

                load data local infile "./opt/caiji/sej001/d5.csv" into table w2 FIELDS TERMINATED BY ',' (id,title,link,comment);
                
运行结束时间：2020-07-28 10:15:31.505864
运行时长：0.004s
影响行数： 23
执行成功！


In [3]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform(onehot_encoded)
print(inverted)

['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


ValueError: bad input shape (10, 3)